# PipeLine
- 데이터를 크롤링하고 정제하여 DB에 저장하는 PipeLine

1. 유저에 대해 DB에 들어갈 내용을 크롤링.

2. 크롤링한 데이터를 정제.

3. 그 내용을 DB에 Insert 또는 Update
-> 기존에 존재하는 value라면, Update,
-> 기존에 존재하지 않는 value라면, Insert

# DB 테이블

1. 유저의 정보를 담은 테이블. - 유저 ID, 유저가 푼 문제 리스트, 유저 Tier, 전체 랭킹(solved.ac 사용하지 않는 사람의 티어를 예측하기 위함), 푼 문제 개수, 시도했지만 맞지 못한 문제 리스트(틀렸습니다는 있는데 맞았습니다는 없는 경우)

2. 문제의 정보를 담은 테이블. - 문제 ID, 문제의 여러 정보들, 문제 context

3. 유저 - 문제 상호작용 테이블. - 유저 ID_문제 ID 키(Insert, Update를 위함), 유저 ID, 문제 ID, 맞은 횟수, 틀린 횟수, 시간 초과, 메모리 초과, 출력 초과, 런타임에러, 컴파일에러, 출력 형식, 그 문제에 대해 가장 많이 사용한 언어, 그 언어 기준 가장 좋은 사용 메모리, 그 언어 기준 가장 적게 걸린 시간, 메모리 제한 최적 풀이의 제출 번호, 시간 제한 최적 풀이의 제출 번호

4. 유저 모델 예측 결과 테이블 - 유저 ID, 유저 취약 알고리즘, FFM 모델 결과 추천 리스트

위의 PipeLine에서는 1,2,3 을 갱신하는 것을 목표로 한다.


In [2]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re
from sklearn.preprocessing import LabelEncoder
import joblib
from collections import Counter
from collections import defaultdict
import ast
import mysql.connector

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
data_path = '../DataSet/'

user_df = pd.read_csv(os.path.join(data_path, 'Baekjoon_유저_크롤링.csv'))
problem_df = pd.read_csv(os.path.join(data_path, 'final_DB_problem_table.csv'))
user_solved_ac_df = pd.read_csv(os.path.join(data_path, 'solved.ac_유저_크롤링.csv'))
user_solved_problem_df = pd.read_csv(os.path.join(data_path, 'Baekjoon_유저_푼_문제_크롤링.csv'))

# 기존 문제 데이터 DB에 집어넣기.

In [122]:
problem_df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
0,1000,A+B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.
1,1001,A-B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.
2,1002,터렛,Silver III,"['수학', '기하학', '많은 조건 분기']",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...
3,1003,피보나치 함수,Silver III,['다이나믹 프로그래밍'],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...
4,1004,어린 왕자,Silver III,"['수학', '기하학']",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,31266,축구 대회,Platinum V,"['다이나믹 프로그래밍', '그리디 알고리즘', '브루트포스 알고리즘', '정렬',...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.
9492,31267,시간 외 근무 멈춰!!!,Platinum III,"['그리디 알고리즘', '이분 탐색', '매개 변수 탐색']",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...
9493,31268,물자 조달,Platinum III,"['그래프 이론', '데이크스트라', '최단 경로', '오프라인 쿼리', '플로이드...",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.
9494,31283,평균 최대화,Ruby V,"['자료 구조', '기하학', '트리', '세그먼트 트리', '이분 탐색', '우선...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.


In [123]:
problem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9496 entries, 0 to 9495
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   problem_number                   9496 non-null   int64  
 1   problem_name                     9496 non-null   object 
 2   problem_tier                     9496 non-null   object 
 3   problem_algorithm                9496 non-null   object 
 4   problem_time_condition           9496 non-null   float64
 5   problem_memory_condition         9496 non-null   int64  
 6   problem_submission_count         9496 non-null   int64  
 7   problem_answer_submission_count  9496 non-null   int64  
 8   problem_answered_people_count    9496 non-null   int64  
 9   problem_correct_rate             9496 non-null   float64
 10  problem_description              9496 non-null   object 
 11  problem_input                    9496 non-null   object 
 12  problem_output      

# Not ratable, Zero 난이도 문제 제외

In [124]:
problem_df = problem_df[problem_df['problem_tier']!='Zero']
problem_df = problem_df[problem_df['problem_tier']!='Not ratable']
problem_df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
0,1000,A+B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.
1,1001,A-B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.
2,1002,터렛,Silver III,"['수학', '기하학', '많은 조건 분기']",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...
3,1003,피보나치 함수,Silver III,['다이나믹 프로그래밍'],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...
4,1004,어린 왕자,Silver III,"['수학', '기하학']",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,31266,축구 대회,Platinum V,"['다이나믹 프로그래밍', '그리디 알고리즘', '브루트포스 알고리즘', '정렬',...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.
9492,31267,시간 외 근무 멈춰!!!,Platinum III,"['그리디 알고리즘', '이분 탐색', '매개 변수 탐색']",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...
9493,31268,물자 조달,Platinum III,"['그래프 이론', '데이크스트라', '최단 경로', '오프라인 쿼리', '플로이드...",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.
9494,31283,평균 최대화,Ruby V,"['자료 구조', '기하학', '트리', '세그먼트 트리', '이분 탐색', '우선...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.


# 구데기 컵 문제들 모두 제외

In [125]:
구데기_list = [i for i in range(15629, 15644)] + [i for i in range(17106, 17121)] + [i for i in range(18821, 18837)] + [i for i in range(21292, 21300)] + [i for i in range(24900, 24912)]
구데기_list

[15629,
 15630,
 15631,
 15632,
 15633,
 15634,
 15635,
 15636,
 15637,
 15638,
 15639,
 15640,
 15641,
 15642,
 15643,
 17106,
 17107,
 17108,
 17109,
 17110,
 17111,
 17112,
 17113,
 17114,
 17115,
 17116,
 17117,
 17118,
 17119,
 17120,
 18821,
 18822,
 18823,
 18824,
 18825,
 18826,
 18827,
 18828,
 18829,
 18830,
 18831,
 18832,
 18833,
 18834,
 18835,
 18836,
 21292,
 21293,
 21294,
 21295,
 21296,
 21297,
 21298,
 21299,
 24900,
 24901,
 24902,
 24903,
 24904,
 24905,
 24906,
 24907,
 24908,
 24909,
 24910,
 24911]

In [126]:
len(구데기_list)

66

In [130]:
problem_df = problem_df[~problem_df['problem_number'].isin(구데기_list)].reset_index(drop=True)

In [131]:
problem_df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
0,1000,A+B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.
1,1001,A-B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.
2,1002,터렛,Silver III,"['수학', '기하학', '많은 조건 분기']",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...
3,1003,피보나치 함수,Silver III,['다이나믹 프로그래밍'],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...
4,1004,어린 왕자,Silver III,"['수학', '기하학']",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9024,31266,축구 대회,Platinum V,"['다이나믹 프로그래밍', '그리디 알고리즘', '브루트포스 알고리즘', '정렬',...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.
9025,31267,시간 외 근무 멈춰!!!,Platinum III,"['그리디 알고리즘', '이분 탐색', '매개 변수 탐색']",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...
9026,31268,물자 조달,Platinum III,"['그래프 이론', '데이크스트라', '최단 경로', '오프라인 쿼리', '플로이드...",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.
9027,31283,평균 최대화,Ruby V,"['자료 구조', '기하학', '트리', '세그먼트 트리', '이분 탐색', '우선...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.


# 정확한 기준이 없는 문제들 처리.

1. ~점 처리

In [54]:
user_df

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
0,66660086,teddybear1024,12920,맞았습니다!!,60700,72,C++17
1,66328551,teddybear1024,29768,맞았습니다!!,2020,0,C++17
2,66328354,teddybear1024,29767,맞았습니다!!,4364,96,C++17
3,66328329,teddybear1024,29767,틀렸습니다,-1,-1,C++17
4,65610566,teddybear1024,13511,맞았습니다!!,47144,244,C++17
...,...,...,...,...,...,...,...
15292182,47690676,kimeunnnn,1697,런타임 에러 (OutOfBounds),-1,-1,C++17
15292183,47690118,kimeunnnn,1389,맞았습니다!!,2064,0,C++17
15292184,47519695,kimeunnnn,1012,맞았습니다!!,2040,0,C++17
15292185,47519665,kimeunnnn,1012,컴파일 에러,-1,-1,C++17


In [55]:
user_df.drop_duplicates('problem_number_list')

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
0,66660086,teddybear1024,12920,맞았습니다!!,60700,72,C++17
1,66328551,teddybear1024,29768,맞았습니다!!,2020,0,C++17
2,66328354,teddybear1024,29767,맞았습니다!!,4364,96,C++17
4,65610566,teddybear1024,13511,맞았습니다!!,47144,244,C++17
6,65273386,teddybear1024,29197,틀렸습니다,-1,-1,C++17
...,...,...,...,...,...,...,...
15068157,72243692,mthw261,31248,맞았습니다!!,31120,272,Python 3
15068595,57288111,mthw261,27872,틀렸습니다,-1,-1,C++17
15111057,70122111,herong2,30880,틀렸습니다,-1,-1,C++20
15111074,69970062,herong2,30872,맞았습니다!!,39340,448,C++20


In [56]:
user_df = user_df[user_df['problem_result_list']!='채점 불가']
user_df = user_df[user_df['problem_result_list']!='채점 중']
user_df

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
0,66660086,teddybear1024,12920,맞았습니다!!,60700,72,C++17
1,66328551,teddybear1024,29768,맞았습니다!!,2020,0,C++17
2,66328354,teddybear1024,29767,맞았습니다!!,4364,96,C++17
3,66328329,teddybear1024,29767,틀렸습니다,-1,-1,C++17
4,65610566,teddybear1024,13511,맞았습니다!!,47144,244,C++17
...,...,...,...,...,...,...,...
15292182,47690676,kimeunnnn,1697,런타임 에러 (OutOfBounds),-1,-1,C++17
15292183,47690118,kimeunnnn,1389,맞았습니다!!,2064,0,C++17
15292184,47519695,kimeunnnn,1012,맞았습니다!!,2040,0,C++17
15292185,47519665,kimeunnnn,1012,컴파일 에러,-1,-1,C++17


In [57]:
a = user_df[user_df['problem_result_list'].str.contains('점')]

In [58]:
a

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
96,61626893,teddybear1024,15824,250점,6708,92,C++17
102,61518714,teddybear1024,25405,100점,6712,308,C++17
103,61518441,teddybear1024,25405,14점,6712,104,C++17
104,61518379,teddybear1024,25405,46점,6712,116,C++17
127,61029244,teddybear1024,27878,70점,5920,2788,C++17
...,...,...,...,...,...,...,...
15291600,72209762,kimeunnnn,17615,15점,12888,108,Java 8
15291788,65382033,kimeunnnn,10162,100점,18720,248,Java 11
15291842,64296410,kimeunnnn,13300,100점,21972,288,Java 11
15291843,64296012,kimeunnnn,10163,100점,23524,280,Java 11


In [59]:
a.drop_duplicates('problem_number_list')

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
96,61626893,teddybear1024,15824,250점,6708,92,C++17
102,61518714,teddybear1024,25405,100점,6712,308,C++17
127,61029244,teddybear1024,27878,70점,5920,2788,C++17
137,60674379,teddybear1024,10166,100점,17660,132,C++17
141,60670044,teddybear1024,21600,150점,2020,20,C++17
...,...,...,...,...,...,...,...
13898821,72120063,tkvl94,31263,64점,2024,0,C++17
14032470,72124874,ohwphil,25433,100점,2544,196,C++17
14034685,67599186,ohwphil,16141,100점,3288,316,C++17
14891547,34571010,opencluster,20194,100점,33272,344,C++17


In [60]:
a['new'] = a['problem_result_list'].str.extract('(\d+\.\d+|\d+)').astype(float)

C:\Users\win10KDH\AppData\Local\Temp\ipykernel_5744\1458893922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['new'] = a['problem_result_list'].str.extract('(\d+\.\d+|\d+)').astype(float)


In [61]:
a

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list,new
96,61626893,teddybear1024,15824,250점,6708,92,C++17,250.0
102,61518714,teddybear1024,25405,100점,6712,308,C++17,100.0
103,61518441,teddybear1024,25405,14점,6712,104,C++17,14.0
104,61518379,teddybear1024,25405,46점,6712,116,C++17,46.0
127,61029244,teddybear1024,27878,70점,5920,2788,C++17,70.0
...,...,...,...,...,...,...,...,...
15291600,72209762,kimeunnnn,17615,15점,12888,108,Java 8,15.0
15291788,65382033,kimeunnnn,10162,100점,18720,248,Java 11,100.0
15291842,64296410,kimeunnnn,13300,100점,21972,288,Java 11,100.0
15291843,64296012,kimeunnnn,10163,100점,23524,280,Java 11,100.0


In [49]:
a.drop_duplicates('problem_number_list')

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list,new
96,61626893,teddybear1024,15824,250점,6708,92,C++17,250.0
102,61518714,teddybear1024,25405,100점,6712,308,C++17,100.0
127,61029244,teddybear1024,27878,70점,5920,2788,C++17,70.0
137,60674379,teddybear1024,10166,100점,17660,132,C++17,100.0
141,60670044,teddybear1024,21600,150점,2020,20,C++17,150.0
...,...,...,...,...,...,...,...,...
13898821,72120063,tkvl94,31263,64점,2024,0,C++17,64.0
14032470,72124874,ohwphil,25433,100점,2544,196,C++17,100.0
14034685,67599186,ohwphil,16141,100점,3288,316,C++17,100.0
14891547,34571010,opencluster,20194,100점,33272,344,C++17,100.0


In [41]:
k = a.groupby('problem_number_list')['new'].max()

In [42]:
k

problem_number_list
2545       3.0
3090     100.0
5522     100.0
5523     100.0
5524     100.0
         ...  
30090    100.0
30091     75.0
31067    100.0
31073     38.0
31263     64.0
Name: new, Length: 435, dtype: float64

In [43]:
k = k.to_dict()
k

{2545: 3.0,
 3090: 100.0,
 5522: 100.0,
 5523: 100.0,
 5524: 100.0,
 5525: 100.0,
 5812: 100.0,
 5814: 100.0,
 5819: 100.0,
 5820: 100.0,
 5821: 100.0,
 5822: 100.0,
 5823: 100.0,
 8872: 100.0,
 8875: 100.0,
 8979: 100.0,
 8980: 100.0,
 8981: 100.0,
 8982: 100.0,
 8983: 100.0,
 8984: 100.0,
 8985: 100.0,
 8986: 100.0,
 8987: 100.0,
 10070: 100.0,
 10072: 20.0,
 10073: 35.0,
 10074: 45.0,
 10075: 100.0,
 10093: 100.0,
 10096: 100.0,
 10162: 100.0,
 10163: 100.0,
 10164: 100.0,
 10165: 100.0,
 10166: 100.0,
 10167: 100.0,
 10168: 100.0,
 10169: 100.0,
 10227: 100.0,
 10713: 100.0,
 10714: 100.0,
 10715: 100.0,
 10717: 100.0,
 10755: 19.0,
 10756: 100.0,
 10833: 100.0,
 10834: 100.0,
 10835: 100.0,
 10836: 100.0,
 10837: 100.0,
 10840: 100.0,
 10841: 100.0,
 10842: 100.0,
 10919: 100.0,
 10922: 100.0,
 10923: 54.0,
 10924: 46.0,
 11933: 100.0,
 11941: 100.0,
 11985: 100.0,
 12095: 10000000.0,
 12752: 100.0,
 13300: 100.0,
 13301: 100.0,
 13302: 100.0,
 13303: 100.0,
 13304: 100.0,
 13305:

In [47]:
len(k.keys())

435

In [53]:
sorted(k.items(),key = lambda x:x[1], reverse=True)

[(29008, 250000000.0),
 (29007, 50000000.0),
 (12095, 10000000.0),
 (27907, 27907.0),
 (18789, 10018.0),
 (15792, 2000.0),
 (16428, 630.0),
 (16682, 600.0),
 (16684, 600.0),
 (16972, 566.39),
 (16679, 400.0),
 (21607, 400.0),
 (15826, 300.0),
 (15827, 300.0),
 (15824, 250.0),
 (15825, 250.0),
 (15831, 250.0),
 (29696, 250.0),
 (29697, 250.0),
 (15822, 200.0),
 (15823, 200.0),
 (21601, 200.0),
 (21606, 200.0),
 (29695, 200.0),
 (20314, 160.0),
 (15830, 150.0),
 (16858, 150.0),
 (21599, 150.0),
 (21600, 150.0),
 (21604, 150.0),
 (21605, 150.0),
 (22032, 150.0),
 (29694, 150.0),
 (17224, 140.0),
 (17225, 140.0),
 (17226, 140.0),
 (17227, 140.0),
 (17228, 140.0),
 (17230, 140.0),
 (17232, 140.0),
 (17233, 140.0),
 (20313, 140.0),
 (22973, 125.0),
 (22974, 125.0),
 (22975, 125.0),
 (22976, 125.0),
 (22977, 125.0),
 (22978, 125.0),
 (22980, 125.0),
 (20312, 120.0),
 (18348, 111.0),
 (3090, 100.0),
 (5522, 100.0),
 (5523, 100.0),
 (5524, 100.0),
 (5525, 100.0),
 (5812, 100.0),
 (5814, 100.0),

In [67]:
crwaling_problem_list = list(a.drop_duplicates('problem_number_list')['problem_number_list'].values)
crwaling_problem_list

[15824,
 25405,
 27878,
 10166,
 21600,
 27397,
 25378,
 27907,
 25427,
 25426,
 17228,
 13308,
 19619,
 14867,
 25380,
 13306,
 20188,
 17619,
 10835,
 16139,
 15971,
 8981,
 10836,
 10164,
 8979,
 13301,
 16208,
 15975,
 15973,
 15828,
 15970,
 15967,
 17616,
 13305,
 5525,
 25379,
 10070,
 11985,
 13302,
 11941,
 5822,
 13310,
 5812,
 14869,
 13309,
 17615,
 15976,
 10167,
 16134,
 12095,
 15779,
 8983,
 10713,
 8986,
 8984,
 16682,
 16679,
 16212,
 16132,
 15821,
 15979,
 15966,
 15965,
 8980,
 14865,
 15554,
 15553,
 22976,
 29007,
 29197,
 10165,
 25402,
 20313,
 20192,
 28270,
 21762,
 21761,
 25401,
 20191,
 14868,
 27466,
 27467,
 22342,
 20187,
 10837,
 21758,
 24912,
 21557,
 16136,
 29696,
 28220,
 28219,
 22901,
 28276,
 28267,
 28265,
 16858,
 8872,
 8987,
 25403,
 15968,
 22902,
 27743,
 25922,
 25924,
 22348,
 22344,
 21760,
 15972,
 14864,
 20309,
 3090,
 22345,
 15831,
 20314,
 16209,
 16213,
 16210,
 17431,
 17275,
 17277,
 15933,
 15704,
 15934,
 17126,
 27935,
 279

In [68]:
len(crwaling_problem_list)

435

In [92]:
complete_list = []
re_list = []

In [96]:
# ~점을 포함한 문제들의 만점을 알기 위한 크롤링 


for problem_number in crwaling_problem_list:

    if((problem_number in complete_list)):
        continue

    print(problem_number)

    url = 'https://www.acmicpc.net/status?&problem_id=' + str(problem_number)

    driver = webdriver.Chrome('../Crawling/chromedriver.exe', options=chrome_options)
    driver.get(url)

    try:
        problem_data = driver.find_elements_by_css_selector('#status-table .result-ac')
        print(problem_data[0].text)
        complete_list.append((problem_number, problem_data[0].text))
    except (NoSuchElementException, IndexError):
        re_list.append(problem_number)


15824
250점
25405


C:\Users\win10KDH\AppData\Local\Temp\ipykernel_5744\2095473262.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if((problem_number in complete_list)):


100점
27878
100점
10166
100점
21600
150점
27397
100점
25378
100점
27907
27907점
25427
100점
25426
100점
17228
13308
100점
19619
100점
14867
100점
25380
100점
13306
100점
20188
100점
17619
100점
10835
100점
16139
100점
15971
100점
8981
100점
10836
10164
100점
8979
100점
13301
100점
16208
30점
15975
100점
15973
100점
15828
100점
15970
100점
15967
100점
17616
100점
13305
100점
5525
100점
25379
100점
10070
100점
11985
100점
13302
100점
11941
100점
5822
100점
13310
100점
5812
100점
14869
100점
13309
100점
17615
100점
15976
100점
10167
100점
16134
100점
12095
109870점
15779
100점
8983
100점
10713
100점
8986
100점
8984
100점
16682
600점
16679
400점
16212
25점
16132
100점
15821
100점
15979
3점
15966
100점
15965
100점
8980
100점
14865
100점
15554
100점
15553
100점
22976
125점
29007
50000000점
29197
100점
10165
100점
25402
100점
20313
140점
20192
100점
28270
3점
21762
100점
21761
100점
25401
100점
20191
100점
14868
100점
27466
100점
27467
100점
22342
100점
20187
100점
10837
100점
21758
100점
24912
100점
21557
100점
16136
100점
29696
250점
28220
100점
28219
100점
22901
100점
28276
3점


In [97]:
save_data = {
        'complete_list': complete_list,
        're_list': re_list
    }

with open('problem_list.pkl', 'wb') as f:
    pickle.dump(save_data, f)

In [103]:
complete_list

[(15824, '250점'),
 (25405, '100점'),
 (27878, '100점'),
 (10166, '100점'),
 (21600, '150점'),
 (27397, '100점'),
 (25378, '100점'),
 (27907, '27907점'),
 (25427, '100점'),
 (25426, '100점'),
 (13308, '100점'),
 (19619, '100점'),
 (14867, '100점'),
 (25380, '100점'),
 (13306, '100점'),
 (20188, '100점'),
 (17619, '100점'),
 (10835, '100점'),
 (16139, '100점'),
 (15971, '100점'),
 (8981, '100점'),
 (10164, '100점'),
 (8979, '100점'),
 (13301, '100점'),
 (16208, '30점'),
 (15975, '100점'),
 (15973, '100점'),
 (15828, '100점'),
 (15970, '100점'),
 (15967, '100점'),
 (17616, '100점'),
 (13305, '100점'),
 (5525, '100점'),
 (25379, '100점'),
 (10070, '100점'),
 (11985, '100점'),
 (13302, '100점'),
 (11941, '100점'),
 (5822, '100점'),
 (13310, '100점'),
 (5812, '100점'),
 (14869, '100점'),
 (13309, '100점'),
 (17615, '100점'),
 (15976, '100점'),
 (10167, '100점'),
 (16134, '100점'),
 (12095, '109870점'),
 (15779, '100점'),
 (8983, '100점'),
 (10713, '100점'),
 (8986, '100점'),
 (8984, '100점'),
 (16682, '600점'),
 (16679, '400점'),
 (16212, '25점'

In [108]:
len(complete_list)

422

In [109]:
complete_list.append((17228,'140점'))
complete_list.append((10836,'100점'))
complete_list.append((20193,'100점'))
complete_list.append((15980,'4점'))
complete_list.append((28217,'100점'))
complete_list.append((18160,'100점'))
complete_list.append((28216,'100점'))
complete_list.append((10072,'20점'))
complete_list.append((10833,'100점'))
complete_list.append((18789,'8140점'))
complete_list.append((16856,'150점'))
complete_list.append((5823,'100점'))
complete_list.append((16141,'100점'))

In [104]:
re_list

[17228,
 10836,
 20193,
 15980,
 28217,
 18160,
 28216,
 10072,
 10833,
 18789,
 16856,
 5823,
 16141]

In [111]:
len(complete_list)

435

2. 맞았습니다!! (%d/%d) 처리

In [113]:
b = user_df[user_df['problem_result_list'].str.contains('맞았습니다!! ()')]
b

C:\Users\win10KDH\AppData\Local\Temp\ipykernel_5744\3752349839.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  b = user_df[user_df['problem_result_list'].str.contains('맞았습니다!! ()')]


,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
1442,41363363,vladimir11,4149,맞았습니다!! (161/161),2028,44,C++17
4633,8146653,movie_jo,15624,맞았습니다!! (201/201),12832,0,C99
7350,47379853,mymasterpark2,15624,맞았습니다!! (201/201),9832,8,C++17
11356,41157018,110000110101,15624,맞았습니다!! (201/201),25332,24,C++17
12988,49713638,kdk,20181,맞았습니다!! (66/66),5928,16,C++17
...,...,...,...,...,...,...,...
15252268,60336498,zxvm5962,15624,맞았습니다!! (201/201),8924,12,C++17
15256455,44140973,chanwlee,20164,맞았습니다!! (102/102),30840,72,Python 3
15262467,52085099,hs1144,15624,맞았습니다!! (201/201),70396,416,Python 3
15277645,45539203,yseo19,15624,맞았습니다!! (201/201),14436,12,C++17


In [114]:
b.drop_duplicates('problem_number_list')

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
1442,41363363,vladimir11,4149,맞았습니다!! (161/161),2028,44,C++17
4633,8146653,movie_jo,15624,맞았습니다!! (201/201),12832,0,C99
12988,49713638,kdk,20181,맞았습니다!! (66/66),5928,16,C++17
13010,49198258,kdk,20164,맞았습니다!! (102/102),2028,0,C++17
13449,27531153,kdk,20183,맞았습니다!! (43/43),25500,856,C++17
15406,28381493,crypt9,21277,맞았습니다!! (19/19),2060,176,C++17
15415,28363573,crypt9,21276,맞았습니다!! (19/19),5172,260,C++17
15565,27680989,crypt9,21278,맞았습니다!! (19/19),2260,4,C++17
15569,27678581,crypt9,21279,맞았습니다!! (16/16),14452,100,C++17
16329,24005135,crypt9,20168,맞았습니다!! (18/18),5140,4,C++17


In [120]:
complete_list_2 = list(zip(list(b.drop_duplicates('problem_number_list')['problem_number_list']),[161,201,66,102,43,19,19,19,16,18,16,11,16,39,500,100,24]))
complete_list_2

[(4149, 161),
 (15624, 201),
 (20181, 66),
 (20164, 102),
 (20183, 43),
 (21277, 19),
 (21276, 19),
 (21278, 19),
 (21279, 16),
 (20168, 18),
 (20182, 16),
 (20165, 11),
 (20166, 16),
 (20167, 39),
 (21133, 500),
 (24727, 100),
 (21275, 24)]

3. 정해진 형식이 없는 문제들 처리 완료.

In [132]:
problem_df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
0,1000,A+B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.
1,1001,A-B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.
2,1002,터렛,Silver III,"['수학', '기하학', '많은 조건 분기']",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...
3,1003,피보나치 함수,Silver III,['다이나믹 프로그래밍'],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...
4,1004,어린 왕자,Silver III,"['수학', '기하학']",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9024,31266,축구 대회,Platinum V,"['다이나믹 프로그래밍', '그리디 알고리즘', '브루트포스 알고리즘', '정렬',...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.
9025,31267,시간 외 근무 멈춰!!!,Platinum III,"['그리디 알고리즘', '이분 탐색', '매개 변수 탐색']",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...
9026,31268,물자 조달,Platinum III,"['그래프 이론', '데이크스트라', '최단 경로', '오프라인 쿼리', '플로이드...",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.
9027,31283,평균 최대화,Ruby V,"['자료 구조', '기하학', '트리', '세그먼트 트리', '이분 탐색', '우선...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.


In [153]:
complete_dict = dict(complete_list)
complete_dict

{15824: '250점',
 25405: '100점',
 27878: '100점',
 10166: '100점',
 21600: '150점',
 27397: '100점',
 25378: '100점',
 27907: '27907점',
 25427: '100점',
 25426: '100점',
 13308: '100점',
 19619: '100점',
 14867: '100점',
 25380: '100점',
 13306: '100점',
 20188: '100점',
 17619: '100점',
 10835: '100점',
 16139: '100점',
 15971: '100점',
 8981: '100점',
 10164: '100점',
 8979: '100점',
 13301: '100점',
 16208: '30점',
 15975: '100점',
 15973: '100점',
 15828: '100점',
 15970: '100점',
 15967: '100점',
 17616: '100점',
 13305: '100점',
 5525: '100점',
 25379: '100점',
 10070: '100점',
 11985: '100점',
 13302: '100점',
 11941: '100점',
 5822: '100점',
 13310: '100점',
 5812: '100점',
 14869: '100점',
 13309: '100점',
 17615: '100점',
 15976: '100점',
 10167: '100점',
 16134: '100점',
 12095: '109870점',
 15779: '100점',
 8983: '100점',
 10713: '100점',
 8986: '100점',
 8984: '100점',
 16682: '600점',
 16679: '400점',
 16212: '25점',
 16132: '100점',
 15821: '100점',
 15979: '3점',
 15966: '100점',
 15965: '100점',
 8980: '100점',
 14865: '100점',


In [154]:
#예외 하나 처리
complete_dict[12095] = '100000점'

In [155]:
converted_complete_dict = {}

for key, value in complete_dict.items():
    # '점'을 제거하고, 숫자 부분만 남김
    numeric_value = int(value[:-1])
    converted_complete_dict[key] = numeric_value

converted_complete_dict

{15824: 250,
 25405: 100,
 27878: 100,
 10166: 100,
 21600: 150,
 27397: 100,
 25378: 100,
 27907: 27907,
 25427: 100,
 25426: 100,
 13308: 100,
 19619: 100,
 14867: 100,
 25380: 100,
 13306: 100,
 20188: 100,
 17619: 100,
 10835: 100,
 16139: 100,
 15971: 100,
 8981: 100,
 10164: 100,
 8979: 100,
 13301: 100,
 16208: 30,
 15975: 100,
 15973: 100,
 15828: 100,
 15970: 100,
 15967: 100,
 17616: 100,
 13305: 100,
 5525: 100,
 25379: 100,
 10070: 100,
 11985: 100,
 13302: 100,
 11941: 100,
 5822: 100,
 13310: 100,
 5812: 100,
 14869: 100,
 13309: 100,
 17615: 100,
 15976: 100,
 10167: 100,
 16134: 100,
 12095: 100000,
 15779: 100,
 8983: 100,
 10713: 100,
 8986: 100,
 8984: 100,
 16682: 600,
 16679: 400,
 16212: 25,
 16132: 100,
 15821: 100,
 15979: 3,
 15966: 100,
 15965: 100,
 8980: 100,
 14865: 100,
 15554: 100,
 15553: 100,
 22976: 125,
 29007: 50000000,
 29197: 100,
 10165: 100,
 25402: 100,
 20313: 140,
 20192: 100,
 28270: 3,
 21762: 100,
 21761: 100,
 25401: 100,
 20191: 100,
 148

In [156]:
complete_dict_2 = dict(complete_list_2)
complete_dict_2

{4149: 161,
 15624: 201,
 20181: 66,
 20164: 102,
 20183: 43,
 21277: 19,
 21276: 19,
 21278: 19,
 21279: 16,
 20168: 18,
 20182: 16,
 20165: 11,
 20166: 16,
 20167: 39,
 21133: 500,
 24727: 100,
 21275: 24}

In [157]:
problem_df['other_result_form_1'] = problem_df['problem_number'].map(converted_complete_dict).fillna(0).astype(int)
problem_df['other_result_form_2'] = problem_df['problem_number'].map(complete_dict_2).fillna(0).astype(int)

In [158]:
problem_df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output,other_result_form_1,other_result_form_2
0,1000,A+B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.,0,0
1,1001,A-B,Bronze V,"['수학', '구현', '사칙연산']",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.,0,0
2,1002,터렛,Silver III,"['수학', '기하학', '많은 조건 분기']",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...,0,0
3,1003,피보나치 함수,Silver III,['다이나믹 프로그래밍'],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...,0,0
4,1004,어린 왕자,Silver III,"['수학', '기하학']",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9024,31266,축구 대회,Platinum V,"['다이나믹 프로그래밍', '그리디 알고리즘', '브루트포스 알고리즘', '정렬',...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.,0,0
9025,31267,시간 외 근무 멈춰!!!,Platinum III,"['그리디 알고리즘', '이분 탐색', '매개 변수 탐색']",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...,0,0
9026,31268,물자 조달,Platinum III,"['그래프 이론', '데이크스트라', '최단 경로', '오프라인 쿼리', '플로이드...",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.,0,0
9027,31283,평균 최대화,Ruby V,"['자료 구조', '기하학', '트리', '세그먼트 트리', '이분 탐색', '우선...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.,0,0


In [161]:
problem_df.isnull().sum()

problem_number                     0
problem_name                       0
problem_tier                       0
problem_algorithm                  0
problem_time_condition             0
problem_memory_condition           0
problem_submission_count           0
problem_answer_submission_count    0
problem_answered_people_count      0
problem_correct_rate               0
problem_description                0
problem_input                      0
problem_output                     0
other_result_form_1                0
other_result_form_2                0
dtype: int64

In [160]:
"""
# 문제 정보 DB에 저장.
import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

create_problem_table_query = "CREATE TABLE IF NOT EXISTS problem_information_table ( \
    problem_number INT PRIMARY KEY, \
    problem_name VARCHAR(1000), \
    problem_tier VARCHAR(30), \
    problem_algorithm VARCHAR(1000), \
    problem_time_condition DOUBLE, \
    problem_memory_condition INT, \
    problem_submission_count INT, \
    problem_answer_submission_count INT, \
    problem_answered_people_count INT, \
    problem_correct_rate DOUBLE, \
    problem_description TEXT, \
    problem_input TEXT, \
    problem_output TEXT, \
    other_result_form_1 INT, \
    other_result_form_2 INT \
);"
cursor.execute(create_problem_table_query)

for index, value in problem_df.iterrows():
    insert_query = "INSERT INTO problem_information_table (problem_number, problem_name, problem_tier, problem_algorithm, problem_time_condition, problem_memory_condition, problem_submission_count, problem_answer_submission_count, problem_answered_people_count, problem_correct_rate, problem_description, problem_input, problem_output, other_result_form_1, other_result_form_2) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
    cursor.execute(insert_query, (value[0], value[1], value[2], value[3], value[4], value[5], value[6], value[7], value[8], value[9], value[10], value[11], value[12], value[13], value[14]))

conn.commit()
cursor.close()
conn.close()

"""
# DB에 문제 정보 저장 완료.

# 기존 유저 데이터 DB에 집어넣기.

In [19]:
user_df.isnull().sum()

submit_number_list           0
user_id_list              8430
problem_number_list       8342
problem_result_list          0
code_memory_list       4009828
code_time_list         4009828
code_language_list        8450
dtype: int64

In [361]:
"""
# 기존 유저 DB Create

import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

create_user_table_query = "CREATE TABLE IF NOT EXISTS user_information_table ( \
    user_id VARCHAR(500) PRIMARY KEY, \
    user_tier INT, \
    user_rank INT, \
    user_solved_problem_list TEXT, \
    user_failed_problem_list TEXT, \
    user_solved_problem_count INT \
);"

create_user_problem_interaction_table_query = "CREATE TABLE IF NOT EXISTS user_problem_interaction_table ( \
    user_problem_key VARCHAR(510) PRIMARY KEY, \
    user_id VARCHAR(500), \
    problem_number INT , \
    correct_attempts_count INT, \
    incorrect_attempts_count INT, \
    time_limit_exceeded_count INT, \
    memory_limit_exceeded_count INT, \
    output_limit_exceeded_count INT, \
    runtime_error_count INT, \
    compile_error_count INT, \
    output_format_issue_count INT, \
    most_used_language VARCHAR(100), \
    best_memory_usage INT, \
    fastest_execution_time INT, \
    memory_limit_optimal_solution_submission_number INT, \
    time_limit_optimal_solution_submission_number INT, \
    FOREIGN KEY (user_id) REFERENCES user_information_table(user_id), \
    FOREIGN KEY (problem_number) REFERENCES problem_information_table(problem_number) \
);"


cursor.execute(create_user_table_query)
cursor.execute(create_user_problem_interaction_table_query)
conn.commit()

cursor.close()
conn.close()
"""

#테이블 틀 생성 완료

In [5]:
import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

set_query = "SET innodb_lock_wait_timeout = 28800;"

cursor.execute(set_query)

In [6]:
problem_number_query = f"SELECT problem_number From problem_information_table"

cursor.execute(problem_number_query)

problem_number_list = [i[0] for i in cursor.fetchall()]
problem_number_list

[1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 1042,
 1043,
 1044,
 1045,
 1046,
 1047,
 1048,
 1049,
 1050,
 1051,
 1052,
 1053,
 1054,
 1055,
 1056,
 1057,
 1058,
 1059,
 1060,
 1061,
 1062,
 1063,
 1064,
 1065,
 1066,
 1067,
 1068,
 1069,
 1070,
 1071,
 1072,
 1073,
 1074,
 1075,
 1076,
 1077,
 1078,
 1079,
 1080,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 1089,
 1090,
 1091,
 1092,
 1093,
 1094,
 1095,
 1096,
 1097,
 1098,
 1099,
 1100,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1110,
 1111,
 1112,
 1113,
 1114,
 1115,
 1116,
 1117,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1137,
 1138,
 1139,
 1140,
 1141,
 1142,

In [284]:
print(len(problem_number_list))

9029


In [7]:
problem_other_result_form_query = f'SELECT other_result_form_1, other_result_form_2 FROM problem_information_table'

cursor.execute(problem_other_result_form_query)
problem_other_result_form = [(i[0],i[1]) for i in cursor.fetchall()]

problem_other_result_form_1 = [int(i[0]) for i in problem_other_result_form]
problem_other_result_form_2 = [int(i[1]) for i in problem_other_result_form]

In [8]:
problem_other_result_form_list_1 = []
problem_other_result_form_list_2 = []

for i, v in enumerate(problem_number_list):
    
    if(problem_other_result_form_1[i] != 0):
        problem_other_result_form_list_1.append((v, problem_other_result_form_1[i]))
    if(problem_other_result_form_2[i] != 0):
        problem_other_result_form_list_2.append((v, problem_other_result_form_2[i]))

problem_other_result_form_dict_1 = dict(problem_other_result_form_list_1)
problem_other_result_form_dict_2 = dict(problem_other_result_form_list_2)

In [268]:
problem_other_result_form_dict_2

{4149: 161,
 15624: 201,
 20164: 102,
 20165: 11,
 20166: 16,
 20167: 39,
 20168: 18,
 20181: 66,
 20182: 16,
 20183: 43,
 21133: 500,
 21275: 24,
 21276: 19,
 21277: 19,
 21278: 19,
 21279: 16,
 24727: 100}

In [9]:
# 유저 아이디 목록이 존재하지 않는 것 존재 -> 데이터 오류, 삭제 요망
user_df = user_df[user_df['user_id_list'] == user_df['user_id_list']]
user_df = user_df[user_df['code_language_list'] == user_df['code_language_list']]

# Problem number 형 변환
# memory, time은 틀렸을 시 nan값 -> 결측치는 아니므로 -1 로 대체

user_df['problem_number_list'] = user_df['problem_number_list'].astype(int)
user_df['code_memory_list'].fillna(-1, inplace=True)
user_df['code_time_list'].fillna(-1, inplace=True)
user_df['code_memory_list'] = user_df['code_memory_list'].astype(int)
user_df['code_time_list'] = user_df['code_time_list'].astype(int)

user_df = user_df[user_df['problem_result_list']!='채점 불가']
user_df = user_df[user_df['problem_result_list']!='채점 중']

#DB에 없는 문제 거르는 작업
user_df = user_df[user_df['problem_number_list'].isin(problem_number_list)]

In [10]:
user_df

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
0,66660086,teddybear1024,12920,맞았습니다!!,60700,72,C++17
1,66328551,teddybear1024,29768,맞았습니다!!,2020,0,C++17
2,66328354,teddybear1024,29767,맞았습니다!!,4364,96,C++17
3,66328329,teddybear1024,29767,틀렸습니다,-1,-1,C++17
4,65610566,teddybear1024,13511,맞았습니다!!,47144,244,C++17
...,...,...,...,...,...,...,...
15292182,47690676,kimeunnnn,1697,런타임 에러 (OutOfBounds),-1,-1,C++17
15292183,47690118,kimeunnnn,1389,맞았습니다!!,2064,0,C++17
15292184,47519695,kimeunnnn,1012,맞았습니다!!,2040,0,C++17
15292185,47519665,kimeunnnn,1012,컴파일 에러,-1,-1,C++17


In [257]:
user_table = user_df.drop_duplicates(['user_id_list']).loc[:,['user_id_list']]
user_table

,user_id_list
0,teddybear1024
823,vladimir11
2364,minkim3987
2918,movie_jo
6666,mymasterpark2
...,...
15288478,hwykyk
15289308,jiho8345
15289836,kchang6869
15290596,khcho0126


In [258]:
user_table.isnull().sum()

user_id_list    0
dtype: int64

In [259]:
user_table = pd.merge(user_table, user_solved_ac_df[['user_name', 'user_tier']], how='inner', left_on='user_id_list', right_on = 'user_name').drop(['user_id_list'],axis=1)
user_table

,user_name,user_tier
0,teddybear1024,21
1,vladimir11,21
2,minkim3987,21
3,movie_jo,21
4,mymasterpark2,21
...,...,...
15418,hwykyk,14
15419,jiho8345,14
15420,kchang6869,14
15421,khcho0126,14


In [260]:
user_table = pd.merge(user_table, user_solved_problem_df, how='inner', left_on='user_name', right_on = 'user_id').drop(['user_id'],axis=1)
user_table

,user_name,user_tier,user_solve_problem
0,teddybear1024,21,"['1000', '1001', '1003', '1005', '1008', '1009..."
1,vladimir11,21,"['1000', '1001', '1002', '1003', '1005', '1007..."
2,minkim3987,21,"['1000', '1001', '1002', '1003', '1004', '1005..."
3,movie_jo,21,"['1000', '1001', '1002', '1003', '1004', '1005..."
4,mymasterpark2,21,"['1000', '1001', '1002', '1003', '1004', '1005..."
...,...,...,...
15418,hwykyk,14,"['1000', '1001', '1002', '1003', '1004', '1005..."
15419,jiho8345,14,"['1000', '1001', '1002', '1008', '1021', '1049..."
15420,kchang6869,14,"['1000', '1001', '1002', '1003', '1004', '1008..."
15421,khcho0126,14,"['1000', '1001', '1003', '1008', '1009', '1012..."


In [261]:
user_table = user_table[user_table['user_solve_problem']!='해당 유저가 존재하지 않습니다.']

In [262]:
user_table['user_solved_problem_count'] = user_table['user_solve_problem'].apply(lambda x: len(ast.literal_eval(x)))
user_table

C:\Users\win10KDH\AppData\Local\Temp\ipykernel_5744\1106440696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_table['user_solved_problem_count'] = user_table['user_solve_problem'].apply(lambda x: len(ast.literal_eval(x)))


,user_name,user_tier,user_solve_problem,user_solved_problem_count
0,teddybear1024,21,"['1000', '1001', '1003', '1005', '1008', '1009...",756
1,vladimir11,21,"['1000', '1001', '1002', '1003', '1005', '1007...",1274
2,minkim3987,21,"['1000', '1001', '1002', '1003', '1004', '1005...",234
3,movie_jo,21,"['1000', '1001', '1002', '1003', '1004', '1005...",3285
4,mymasterpark2,21,"['1000', '1001', '1002', '1003', '1004', '1005...",1326
...,...,...,...,...
15418,hwykyk,14,"['1000', '1001', '1002', '1003', '1004', '1005...",182
15419,jiho8345,14,"['1000', '1001', '1002', '1008', '1021', '1049...",203
15420,kchang6869,14,"['1000', '1001', '1002', '1003', '1004', '1008...",343
15421,khcho0126,14,"['1000', '1001', '1003', '1008', '1009', '1012...",278


In [263]:
c = Counter(list(user_df['problem_result_list']))
c

Counter({'맞았습니다!!': 6528352,
         '틀렸습니다': 4527284,
         '컴파일 에러': 577449,
         '런타임 에러 (OutOfBounds)': 110177,
         '시간 초과': 1511235,
         '250점': 1248,
         '100점': 87075,
         '14점': 294,
         '46점': 52,
         '출력 초과': 169547,
         '70점': 245,
         '150점': 298,
         '6점': 461,
         '907점': 218,
         '140점': 669,
         '메모리 초과': 364845,
         '49점': 310,
         '34점': 622,
         '런타임 에러': 286830,
         '59점': 47,
         '런타임 에러 (DoubleFree)': 3370,
         '런타임 에러 (DivisionByZero)': 6679,
         '출력 형식이 잘못되었습니다': 43552,
         '33점': 669,
         '30점': 2836,
         '4점': 634,
         '21점': 673,
         '런타임 에러 (IntegerOverflow)': 6934,
         '31점': 683,
         '런타임 에러 (Segfault)': 33547,
         '15점': 2939,
         '맞았습니다!! (161/161)': 485,
         '12점': 605,
         '런타임 에러 (out_of_range)': 4109,
         '런타임 에러 (invalid_argument)': 1026,
         '런타임 에러 (AccessNullPointer)': 1846,
      

In [264]:
c.keys()

dict_keys(['맞았습니다!!', '틀렸습니다', '컴파일 에러', '런타임 에러 (OutOfBounds)', '시간 초과', '250점', '100점', '14점', '46점', '출력 초과', '70점', '150점', '6점', '907점', '140점', '메모리 초과', '49점', '34점', '런타임 에러', '59점', '런타임 에러 (DoubleFree)', '런타임 에러 (DivisionByZero)', '출력 형식이 잘못되었습니다', '33점', '30점', '4점', '21점', '런타임 에러 (IntegerOverflow)', '31점', '런타임 에러 (Segfault)', '15점', '맞았습니다!! (161/161)', '12점', '런타임 에러 (out_of_range)', '런타임 에러 (invalid_argument)', '런타임 에러 (AccessNullPointer)', '런타임 에러 (WithoutReturning)', '런타임 에러 (InsufficientSpace)', '50점', '런타임 에러 (IndexError)', '9점', '62점', '90점', '런타임 에러 (IndexOutOfRange)', '109855점', '109919점', '런타임 에러 (DivideByZero)', '런타임 에러 (NullPointer)', '600점', '400점', '25점', '19점', '607점', '3점', '11점', '맞았습니다!! (201/201)', '런타임 에러 (NZEC)', '125점', '런타임 에러 (ShiftExponent)', '50000000점', '1점', '17점', '56점', '23점', '41점', '29점', '런타임 에러 (bad_alloc)', '런타임 에러 (파일 생성)', '63점', '20점', '8점', '58점', '24점', '런타임 에러 (InvalidPointer)', '52점', '2점', '47점', '10점', '85점', '런타임 에러 (InvalidNex

In [265]:
len(c.keys())

1582

In [266]:
result_list = list(user_df['problem_result_list'].unique())
result = set()

pattern = re.compile(r'^\d+(\.\d+)?점$')

for i in result_list:
    if(('맞았습니다!!' in i) or pattern.match(i)):
        result.add('맞았습니다!!')
    elif('런타임 에러' in i):
        result.add('런타임 에러')
    elif('틀렸습니다' in i):
        result.add('틀렸습니다')
    else:
        result.add(i)
result
    

{'런타임 에러',
 '맞았습니다!!',
 '메모리 초과',
 '시간 초과',
 '출력 초과',
 '출력 형식이 잘못되었습니다',
 '컴파일 에러',
 '틀렸습니다'}

In [363]:
complete_insert_list = []

In [364]:
user_name_list = list(user_table['user_name'])
len_user_name_list = len(user_name_list)

other_form_1_key = problem_other_result_form_dict_1.keys()
other_form_2_key = problem_other_result_form_dict_2.keys()

for index, user_id in enumerate(user_name_list):
    
    if(user_id in complete_insert_list):
        continue

    print(f'{user_id} ({index}/{len_user_name_list})...')

    user_id_insert_query = "INSERT INTO user_information_table (user_id) VALUES (%s);"
    cursor.execute(user_id_insert_query, (user_id,))
    conn.commit()

    user_failed_problem = []
    
    sub_df = user_df[user_df['user_id_list']==user_id]

    g_sub_df = sub_df.groupby('problem_number_list')
    
    for problem_number, row in g_sub_df:
        
        result_dict = {
            'user_problem_key' : 0,
            'user_id' : 0,
            'problem_number' : 0,
            'correct_attempts_count' : 0,
            'incorrect_attempts_count' : 0,
            'time_limit_exceeded_count' : 0,
            'memory_limit_exceeded_count' : 0,
            'output_limit_exceeded_count' : 0,
            'runtime_error_count' : 0,
            'compile_error_count' : 0,
            'output_format_issue_count' : 0,
            'most_used_language' : 0,
            'best_memory_usage' : 0,
            'fastest_execution_time' : 0,
            'memory_limit_optimal_solution_submission_number' : 0,
            'time_limit_optimal_solution_submission_number' : 0
        }

        correct_attempts_list = []

        result_dict['user_problem_key'] = str(user_id) + '_' + str(problem_number)
        result_dict['user_id'] = user_id
        result_dict['problem_number'] = problem_number
        
        for index, r in row.iterrows():

            flag = False
            result = r['problem_result_list']

            if(problem_number in other_form_1_key):
                if('점' in result):
                    result_score = re.search(r'\d+(\.\d+)?', result).group()
                    if(float(result_score) if '.' in result_score else int(result_score) == problem_other_result_form_dict_1[problem_number]):
                        result_dict['correct_attempts_count'] += 1
                        flag = True
                    else:
                        result_dict['incorrect_attempts_count'] += 1
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass
            elif(problem_number in other_form_2_key):
                if('맞았습니다!!' in result):
                    numbers = re.findall(r'\d+', result)
                    if(numbers[0] == numbers[1]):
                        result_dict['correct_attempts_count'] += 1
                        flag = True
                    elif(numbers[0] != numbers[1]):
                        result_dict['incorrect_attempts_count'] += 1
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass
            else:
                if('맞았습니다!!' in result):
                    result_dict['correct_attempts_count'] += 1
                    flag = True
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass

            language = r['code_language_list']

            if(flag):
                correct_attempts_list.append((r['submit_number_list'], r['code_memory_list'], r['code_time_list'], r['code_language_list']))

        if(result_dict['correct_attempts_count']==0):
            user_failed_problem.append(problem_number)
        
        else:
            frequency_counter = Counter(tuple_[3] for tuple_ in correct_attempts_list)
            most_common_language = frequency_counter.most_common(1)[0][0]
            filtered_tuples = [tuple_ for tuple_ in correct_attempts_list if tuple_[3] == most_common_language]

            best_memory = min(filtered_tuples, key=lambda tuple_: tuple_[1])
            best_time = min(filtered_tuples, key=lambda tuple_: tuple_[2])

            result_dict['most_used_language'] = most_common_language
            result_dict['best_memory_usage'] = best_memory[1]
            result_dict['fastest_execution_time'] = best_time[2]
            result_dict['memory_limit_optimal_solution_submission_number'] = best_memory[0]
            result_dict['time_limit_optimal_solution_submission_number'] = best_time[0]

        user_problem_interaction_insert_query = "INSERT INTO user_problem_interaction_table (user_problem_key ,user_id ,problem_number ,correct_attempts_count ,incorrect_attempts_count ,time_limit_exceeded_count ,memory_limit_exceeded_count ,output_limit_exceeded_count ,runtime_error_count ,compile_error_count ,output_format_issue_count ,most_used_language ,best_memory_usage ,fastest_execution_time ,memory_limit_optimal_solution_submission_number ,time_limit_optimal_solution_submission_number) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(user_problem_interaction_insert_query, (result_dict['user_problem_key'], result_dict['user_id'], result_dict['problem_number'], result_dict['correct_attempts_count'], result_dict['incorrect_attempts_count'], result_dict['time_limit_exceeded_count'], result_dict['memory_limit_exceeded_count'], result_dict['output_limit_exceeded_count'], result_dict['runtime_error_count'], result_dict['compile_error_count'], result_dict['output_format_issue_count'], result_dict['most_used_language'], result_dict['best_memory_usage'], result_dict['fastest_execution_time'], result_dict['memory_limit_optimal_solution_submission_number'], result_dict['time_limit_optimal_solution_submission_number']))
        conn.commit()
    
    user_table_update_query = "UPDATE user_information_table SET user_tier = %s, user_solved_problem_list = %s, user_failed_problem_list = %s, user_solved_problem_count = %s WHERE user_id = %s;"
    cursor.execute(user_table_update_query, (int(user_table[user_table['user_name']==user_id].loc[:,'user_tier']), user_table[user_table['user_name']==user_id].loc[:,'user_solve_problem'].values[0], str(user_failed_problem), int(user_table[user_table['user_name']==user_id].loc[:,'user_solved_problem_count']), user_id))
    conn.commit()

    complete_insert_list.append(user_id)

    file_name = "complete_insert_user_id.pkl"
    with open(file_name, 'wb') as file:
        pickle.dump(complete_insert_list, file)

cursor.close()
conn.close()

#user_information 먼저 만들고 interaction 테이블 넣어야해.. 외래키 쓰고있잖아

#피클에 저장은 하되... 혹시모르니까 user_table이랑 user_problem_interaction_table이랑 짝이 맞는지 검수해야해. 아니면 삭제해야하거든.

teddybear1024 (0/15416)...
vladimir11 (1/15416)...
minkim3987 (2/15416)...
movie_jo (3/15416)...
mymasterpark2 (4/15416)...
paengjiwon (5/15416)...
sedev57 (6/15416)...
110000110101 (7/15416)...
kdk (8/15416)...
repelit (9/15416)...
crypt9 (10/15416)...
dnlsehdn9779 (11/15416)...
kkpos0206 (12/15416)...
wjshim (13/15416)...
youngjun7 (14/15416)...
ajdxjdrnfl (15/15416)...
songuno1 (16/15416)...
vega (17/15416)...
dnialh (18/15416)...
akswnd98 (19/15416)...
cooki0615 (20/15416)...
tjdgus4384 (21/15416)...
jjmjm2002 (22/15416)...
khcho0125 (23/15416)...
tykr0001 (24/15416)...
Apple_Cplus (25/15416)...
huisung0205 (26/15416)...
solibaht (27/15416)...
mooni (28/15416)...
onteeae (29/15416)...
louisdebroglie (30/15416)...
rasauq1122 (31/15416)...
cptech (32/15416)...
wlgur1023 (33/15416)...
cocoa_chan (34/15416)...
impri (35/15416)...
gs16109 (36/15416)...
poketred12 (37/15416)...
roeyr (38/15416)...
jkuk5046 (39/15416)...
sr3gm (40/15416)...
jpseo99 (41/15416)...
mingyu331 (42/15416)...
Ri

# 유저 데이터 PipeLine 생성

In [1]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re
from sklearn.preprocessing import LabelEncoder
import joblib
from collections import Counter
from collections import defaultdict
import ast
import mysql.connector

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [2]:
import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

set_query = "SET innodb_lock_wait_timeout = 28800;"

cursor.execute(set_query)

In [ ]:
def prepare_problem_list():
    problem_number_query = f"SELECT problem_number From problem_information_table"

    cursor.execute(problem_number_query)

    problem_number_list = [i[0] for i in cursor.fetchall()]

    problem_other_result_form_query = f'SELECT other_result_form_1, other_result_form_2 FROM problem_information_table'

    cursor.execute(problem_other_result_form_query)
    problem_other_result_form = [(i[0],i[1]) for i in cursor.fetchall()]

    problem_other_result_form_1 = [int(i[0]) for i in problem_other_result_form]
    problem_other_result_form_2 = [int(i[1]) for i in problem_other_result_form]

    problem_other_result_form_list_1 = []
    problem_other_result_form_list_2 = []

    for i, v in enumerate(problem_number_list):
        
        if(problem_other_result_form_1[i] != 0):
            problem_other_result_form_list_1.append((v, problem_other_result_form_1[i]))
        if(problem_other_result_form_2[i] != 0):
            problem_other_result_form_list_2.append((v, problem_other_result_form_2[i]))

    problem_other_result_form_dict_1 = dict(problem_other_result_form_list_1)
    problem_other_result_form_dict_2 = dict(problem_other_result_form_list_2)

    return (problem_number_list, problem_other_result_form_dict_1, problem_other_result_form_dict_2)

In [ ]:
def crawling_user_information(user_id, problem_number_list):

    user_information_url = 'https://www.acmicpc.net/user/' + str(user_id)

    driver = webdriver.Chrome('../Crawling/chromedriver.exe', options=chrome_options)
    driver.get(user_information_url)


    user_solve_problem_list = []
    tier_value = -1
    rank_value = -1

    #유저 정보가 존재한다면, 유저 정보 긁어오기.
    try:
        user_solve_problem_object = driver.find_element_by_css_selector('.problem-list')
        user_solve_problem = user_solve_problem_object.text.split(' ')
        user_solve_problem_list = [int(problem) for problem in user_solve_problem if int(problem) in problem_number_list]
    
    #존재하지 않는다면, 오류 메세지 출력
    except NoSuchElementException:
        print('해당 유저가 존재하지 않습니다.')
        return -1

    try:
        # id가 "statics"인 테이블 찾기
        table_element = driver.find_element_by_css_selector("#statics")
        
        # 테이블 내에서 '등수' 텍스트를 포함한 <tr> 찾기
        for t in table_element.text.split('\n'):
            if('등수' in t):
                rank_value = int(t.split(' ')[1])
                break

    except NoSuchElementException:
        print('해당 유저의 랭크가 존재하지 않습니다.')

    try:
        solvedac_tier_element = driver.find_element_by_css_selector(".solvedac-tier")
        img_src = solvedac_tier_element.get_attribute("src")

        pattern = r'/tier/(\d+)\.svg'

        match = re.search(pattern, img_src)

        tier_value = int(match.group(1))
    except NoSuchElementException:
        print('해당 유저의 티어가 존재하지 않습니다.')


    driver.quit()

    return (tier_value, rank_value, user_solve_problem_list, len(user_solve_problem_list))

In [ ]:
def crawling_user_problem_interaction(user_id):

    user_problem_interaction_url = 'https://www.acmicpc.net/status?user_id=' + str(user_id)
    driver = webdriver.Chrome('../Crawling/chromedriver.exe', options=chrome_options)
    driver.get(user_problem_interaction_url)

    submit_number_list = []
    user_id_list = []
    problem_number_list = []
    problem_result_list = []
    code_memory_list = []
    code_time_list = []
    code_language_list = []


    flag = True

    while(True):

        user_data = driver.find_elements_by_css_selector('[data-can-view]')
        
        try:
            user_next_page = driver.find_element_by_css_selector('#next_page')

        except NoSuchElementException:
            flag = False

        for user_info in user_data:
        
            user_text = user_info.find_elements_by_css_selector("td")
            user_text_list = []

            for ut in user_text:
                user_text_list.append(ut.text)
            
            submit_number_list.append(user_text_list[0])
            user_id_list.append(user_text_list[1])
            problem_number_list.append(user_text_list[2])
            problem_result_list.append(user_text_list[3])
            code_memory_list.append(np.nan if user_text_list[4]=='' else user_text_list[4])
            code_time_list.append(np.nan if user_text_list[5]=='' else user_text_list[5])
            code_language_list.append(user_text_list[6])

        if(flag):
            user_next_page.click()
        else:
            break            
    
    user_problem_interaction_information = pd.DataFrame({
        'submit_number_list': submit_number_list,
        'user_id_list': user_id_list,
        'problem_number_list': problem_number_list,
        'problem_result_list': problem_result_list,
        'code_memory_list': code_memory_list,
        'code_time_list': code_time_list,
        'code_language_list': code_language_list,
    })

    driver.quit()
    
    return user_problem_interaction_information

In [ ]:
def user_problem_interaction_preprocessing(user_problem_interaction_df, problem_number_list):
    # 유저 아이디 목록이 존재하지 않는 것 존재 -> 데이터 오류, 삭제 요망
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['user_id_list'] == user_problem_interaction_df['user_id_list']]
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['code_language_list'] == user_problem_interaction_df['code_language_list']]

    # Problem number 형 변환
    # memory, time은 틀렸을 시 nan값 -> 결측치는 아니므로 -1 로 대체

    user_problem_interaction_df['problem_number_list'] = user_problem_interaction_df['problem_number_list'].astype(int)
    user_problem_interaction_df['code_memory_list'].fillna(-1, inplace=True)
    user_problem_interaction_df['code_time_list'].fillna(-1, inplace=True)
    user_problem_interaction_df['code_memory_list'] = user_problem_interaction_df['code_memory_list'].astype(int)
    user_problem_interaction_df['code_time_list'] = user_problem_interaction_df['code_time_list'].astype(int)

    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['problem_result_list']!='채점 불가']
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['problem_result_list']!='채점 중']

    #DB에 없는 문제 거르는 작업
    user_problem_interaction_df = user_problem_interaction_df[user_problem_interaction_df['problem_number_list'].isin(problem_number_list)]

    return user_problem_interaction_df

In [ ]:
def user_problem_interaction_to_DB_form(user_id, user_problem_interaction_df, problem_other_result_form_dict_1, problem_other_result_form_dict_2):

    total_result_dict = dict()
    user_failed_problem = []

    user_problem_interaction_df_groupby_problem_number = user_problem_interaction_df.groupby('problem_number_list')

    other_form_1_key = problem_other_result_form_dict_1.keys()
    other_form_2_key = problem_other_result_form_dict_2.keys()

    for problem_number, row in user_problem_interaction_df_groupby_problem_number:
        
        result_dict = {
            'user_problem_key' : 0,
            'user_id' : 0,
            'problem_number' : 0,
            'correct_attempts_count' : 0,
            'incorrect_attempts_count' : 0,
            'time_limit_exceeded_count' : 0,
            'memory_limit_exceeded_count' : 0,
            'output_limit_exceeded_count' : 0,
            'runtime_error_count' : 0,
            'compile_error_count' : 0,
            'output_format_issue_count' : 0,
            'most_used_language' : 0,
            'best_memory_usage' : 0,
            'fastest_execution_time' : 0,
            'memory_limit_optimal_solution_submission_number' : 0,
            'time_limit_optimal_solution_submission_number' : 0
        }

        correct_attempts_list = []

        result_dict['user_problem_key'] = str(user_id) + '_' + str(problem_number)
        result_dict['user_id'] = user_id
        result_dict['problem_number'] = problem_number
        
        for index, r in row.iterrows():

            flag = False
            result = r['problem_result_list']

            if(problem_number in other_form_1_key):
                if('점' in result):
                    result_score = re.search(r'\d+(\.\d+)?', result).group()
                    if(float(result_score) if '.' in result_score else int(result_score) == problem_other_result_form_dict_1[problem_number]):
                        result_dict['correct_attempts_count'] += 1
                        flag = True
                    else:
                        result_dict['incorrect_attempts_count'] += 1
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass
            elif(problem_number in other_form_2_key):
                if('맞았습니다!!' in result):
                    numbers = re.findall(r'\d+', result)
                    if(numbers[0] == numbers[1]):
                        result_dict['correct_attempts_count'] += 1
                        flag = True
                    elif(numbers[0] != numbers[1]):
                        result_dict['incorrect_attempts_count'] += 1
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass
            else:
                if('맞았습니다!!' in result):
                    result_dict['correct_attempts_count'] += 1
                    flag = True
                elif('틀렸습니다' in result):
                    result_dict['incorrect_attempts_count'] += 1
                elif('시간 초과' in result):
                    result_dict['time_limit_exceeded_count'] += 1
                elif('메모리 초과' in result):
                    result_dict['memory_limit_exceeded_count'] += 1
                elif('출력 초과' in result):
                    result_dict['output_limit_exceeded_count'] += 1
                elif('런타임 에러' in result):
                    result_dict['runtime_error_count'] += 1
                elif('컴파일 에러' in result):
                    result_dict['compile_error_count'] += 1
                elif('출력 형식이 잘못되었습니다' in result):
                    result_dict['output_format_issue_count'] += 1
                else:
                    pass

            if(flag):
                correct_attempts_list.append((r['submit_number_list'], r['code_memory_list'], r['code_time_list'], r['code_language_list']))

        if(result_dict['correct_attempts_count']==0):
            user_failed_problem.append(problem_number)
        
        else:
            frequency_counter = Counter(tuple_[3] for tuple_ in correct_attempts_list)
            most_common_language = frequency_counter.most_common(1)[0][0]
            filtered_tuples = [tuple_ for tuple_ in correct_attempts_list if tuple_[3] == most_common_language]

            best_memory = min(filtered_tuples, key=lambda tuple_: tuple_[1])
            best_time = min(filtered_tuples, key=lambda tuple_: tuple_[2])

            result_dict['most_used_language'] = most_common_language
            result_dict['best_memory_usage'] = best_memory[1]
            result_dict['fastest_execution_time'] = best_time[2]
            result_dict['memory_limit_optimal_solution_submission_number'] = best_memory[0]
            result_dict['time_limit_optimal_solution_submission_number'] = best_time[0]
        
        total_result_dict[problem_number] = result_dict


    return (total_result_dict, user_failed_problem)
        

In [ ]:
def new_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info):

    #유저 테이블 데이터 전처리 및 DB 저장 함수 실행. (Insert)
    #- 유저 id가 없으므로, Insert 실행

    user_id_insert_query = "INSERT INTO user_information_table (user_id, user_tier, user_rank, user_solved_problem_list, user_failed_problem_list, user_solved_problem_count) VALUES (%s, %s, %s, %s, %s, %s);"
    cursor.execute(user_id_insert_query, (user_id, user_tier, user_rank, str(user_solved_problem), str(user_failed_problem), user_solved_problem_number))
    conn.commit()

    #유저 - 문제 상호작용 테이블 데이터 전처리 및 DB 저장 함수 실행. (Insert)
    #- 유저 - 문제 Key에서 유저 자체가 없으므로, 모든 정보 Insert

    user_problem_interaction_insert_query = "INSERT INTO user_problem_interaction_table (user_problem_key ,user_id ,problem_number ,correct_attempts_count ,incorrect_attempts_count ,time_limit_exceeded_count ,memory_limit_exceeded_count ,output_limit_exceeded_count ,runtime_error_count ,compile_error_count ,output_format_issue_count ,most_used_language ,best_memory_usage ,fastest_execution_time ,memory_limit_optimal_solution_submission_number ,time_limit_optimal_solution_submission_number) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    for i, v in user_problem_interaction_info.items():
        cursor.execute(user_problem_interaction_insert_query, (v['user_problem_key'], v['user_id'], v['problem_number'], v['correct_attempts_count'], v['incorrect_attempts_count'], v['time_limit_exceeded_count'], v['memory_limit_exceeded_count'], v['output_limit_exceeded_count'], v['runtime_error_count'], v['compile_error_count'], v['output_format_issue_count'], v['most_used_language'], v['best_memory_usage'], v['fastest_execution_time'], v['memory_limit_optimal_solution_submission_number'], v['time_limit_optimal_solution_submission_number']))
    
    conn.commit()
    

In [ ]:
def regular_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info):

    #유저 테이블 데이터 전처리 및 DB 저장 함수 실행. (Update)
    #- 유저 id 찾아서 업데이트
    
    user_table_update_query = "UPDATE user_information_table SET user_tier = %s, user_rank = %s, user_solved_problem_list = %s, user_failed_problem_list = %s, user_solved_problem_count = %s WHERE user_id = %s;"
    cursor.execute(user_table_update_query, (user_tier, user_rank, str(user_solved_problem), str(user_failed_problem), user_solved_problem_number, user_id))
    conn.commit()

    #유저 - 문제 상호작용 테이블 데이터 전처리 및 DB 저장 함수 실행. (Update)
    #- 유저 - 문제 Key가 있다면, Update 없다면, Insert

    user_problem_interaction_insert_query = "INSERT INTO user_problem_interaction_table (user_problem_key ,user_id ,problem_number ,correct_attempts_count ,incorrect_attempts_count ,time_limit_exceeded_count ,memory_limit_exceeded_count ,output_limit_exceeded_count ,runtime_error_count ,compile_error_count ,output_format_issue_count ,most_used_language ,best_memory_usage ,fastest_execution_time ,memory_limit_optimal_solution_submission_number ,time_limit_optimal_solution_submission_number) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    user_problem_interaction_update_query = "UPDATE user_problem_interaction_table SET correct_attempts_count = %s, incorrect_attempts_count = %s, time_limit_exceeded_count = %s, memory_limit_exceeded_count = %s, output_limit_exceeded_count = %s, runtime_error_count = %s, compile_error_count = %s, output_format_issue_count = %s, most_used_language = %s, best_memory_usage = %s, fastest_execution_time = %s, memory_limit_optimal_solution_submission_number = %s, time_limit_optimal_solution_submission_number = %s WHERE user_problem_key = %s;"              
    
    for i, v in user_problem_interaction_info.items():

        up_key = v['user_problem_key']

        check_user_problem_key_query = f'SELECT EXISTS (SELECT TRUE FROM user_problem_interaction_table WHERE user_problem_key = \'{up_key}\') AS value_exists;'
    
        cursor.execute(check_user_problem_key_query)
        
        check_user_problem_key_result = cursor.fetchone()

        if(check_user_problem_key_result[0] == 1):
            cursor.execute(user_problem_interaction_update_query, (v['correct_attempts_count'], v['incorrect_attempts_count'], v['time_limit_exceeded_count'], v['memory_limit_exceeded_count'], v['output_limit_exceeded_count'], v['runtime_error_count'], v['compile_error_count'], v['output_format_issue_count'], v['most_used_language'], v['best_memory_usage'], v['fastest_execution_time'], v['memory_limit_optimal_solution_submission_number'], v['time_limit_optimal_solution_submission_number'], v['user_problem_key']))
        else:
            cursor.execute(user_problem_interaction_insert_query, (v['user_problem_key'], v['user_id'], v['problem_number'], v['correct_attempts_count'], v['incorrect_attempts_count'], v['time_limit_exceeded_count'], v['memory_limit_exceeded_count'], v['output_limit_exceeded_count'], v['runtime_error_count'], v['compile_error_count'], v['output_format_issue_count'], v['most_used_language'], v['best_memory_usage'], v['fastest_execution_time'], v['memory_limit_optimal_solution_submission_number'], v['time_limit_optimal_solution_submission_number']))
    
    conn.commit()
    

In [ ]:
#렉걸려서 잠시 연결 안된 예외도 처리해주자.

def user_update(user_id):

    problem_number_list, problem_other_result_form_dict_1, problem_other_result_form_dict_2 = prepare_problem_list()
    
    #유저 테이블에 필요한 정보 크롤링하는 함수 실행.
    user_information = crawling_user_information(user_id,problem_number_list)
    
    #유저가 존재하지 않는다면,
    if(user_information == -1):
        print('해당 유저가 존재하지 않습니다.')
        #해당 유저의 데이터 삭제하는 코드 실행.

        #근데, 존재하지 않으면 데이터를 집어넣을 일이 없고, 이미 넣어진 상태에서 제거된다는 것은 유저가 탈퇴한 경우인데, 
        #AI의 인풋 데이터가 될 수도 있는 것을 굳이 삭제할 필요가 있을까?... 

        return -1

    #유저의 제출들을 크롤링하는 함수 실행.
    user_problem_interaction = crawling_user_problem_interaction(user_id)

    # user_problem_interaction 전처리 함수 실행

    user_problem_interaction = user_problem_interaction_preprocessing(user_problem_interaction, problem_number_list)
    user_problem_interaction = user_problem_interaction_to_DB_form(user_id, user_problem_interaction, problem_other_result_form_dict_1, problem_other_result_form_dict_2)
    
    print(user_information)
    print(user_problem_interaction)

    user_tier = user_information[0]
    user_rank = user_information[1]
    user_solved_problem = user_information[2]
    user_solved_problem_number = user_information[3]

    user_problem_interaction_info = user_problem_interaction[0]
    user_failed_problem = user_problem_interaction[1]


    # DB의 유저 테이블에 해당 유저가 존재하는지 확인.

    check_user_id_query = f'SELECT EXISTS (SELECT TRUE FROM user_information_table WHERE user_id = \'{user_id}\') AS value_exists;'
    
    cursor.execute(check_user_id_query)
    
    check_user_id_result = cursor.fetchone()
    

    # 있다면, 해당 유저의 정보를 갱신.
    if(check_user_id_result[0] == 1):
        regular_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info)
        print('기존 유저 갱신')
        
        
    # 없다면, 유저의 정보를 추가
    else:
        new_user_process(user_id, user_tier, user_rank, user_solved_problem, user_failed_problem, user_solved_problem_number, user_problem_interaction_info)
        print('신규 유저 추가')
        
    cursor.close()
    conn.close()